In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os,sys,subprocess
from classy import Class
from scipy.optimize import fsolve
from scipy.interpolate import interp1d, UnivariateSpline, splrep, splev,sproot,PPoly, InterpolatedUnivariateSpline, CubicSpline
from tqdm import trange
import math
from math import pi
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import AutoMinorLocator, LogLocator, NullFormatter
import scienceplots
#plt.style.use('mine.mplstyle')
plt.style.use(['science','nature'])
plt.rcParams["axes.axisbelow"] = False

# Main Settings and Functions 

In [ ]:
common_settings = {'output' :'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':3.0,
                   # LambdaCDM parameters
                   '100*theta_s':1.041783, #We are fixing the value of the sound horizon (not h!)
                   'l_max_scalars':4000, #maximum value of \ell in CMB power spectra
                   'omega_b':0.02238280,
                   'A_s':2.100549e-09,
                   'n_s':0.9660499,
                   'tau_reio':0.05430842,
                   'compute_damping_scale':'yes'#We are computing directly the damping scale in Mpc
                   }

omega_cdm_LCDM=0.1201075 #\omega_cdm=|Omega_cdm*h^2

a_eq=0.000293534798833 #value of a_{eq} for the fiducial model, to be fixed in all models
omega_photon=2.47297928086*10**(-5)
a_nu=8/7*(11/4)**(4/3)

N_eff_array=np.linspace(0.1,7,100)
omega_cdm_array=np.array([omega_cdm_LCDM+1/(a_nu*a_eq)*(i-3.044)*omega_photon for i in N_eff_array])

YHes=np.linspace(0.01,0.5,100)

Neff_less=np.linspace(1, 2.9, num=50)
Neff_more=np.linspace(3.1, 6, num=50)
N_eff_array2=np.zeros(100)
N_eff_array2[:50]=Neff_less
N_eff_array2[50:]=Neff_more
omega_cdms_v2=np.array([omega_cdm_LCDM+1/(a_nu*a_eq)*(i-3.044)*omega_photon for i in N_eff_array2])
YHe_vals=np.zeros(len(N_eff_array2))
thD_fid=0.0016144385060630974


In [ ]:
def Cl_computer(global_settings,specific_cosmo_settings,ell_max=4000):
    conv_factor=(2.726*10**(6))**2/10**(3) #Conversion factor from dimensionless units to 10^3 \muK^2
    cosmo=Class()
    cosmo.set(global_settings)
    cosmo.set(specific_cosmo_settings)
    cosmo.compute()
    cls = cosmo.lensed_cl(ell_max)
    ells = cls['ell'][2:]
    dTTs = cls['tt'][2:]*(ells*(ells+1))/(2*np.pi)*conv_factor
    dEEs = cls['ee'][2:]*ells*(ells+1)/(2*np.pi)*conv_factor
    cosmo.struct_cleanup()
    cosmo.empty()
    return ells,dTTs,dEEs

def Kll(ells,dll,aV,thV,kV):
    return dll*np.exp(aV*(ells*thV)**kV)

def filter_roots(X, dgdx,d2gdx2,distance):
    # Create a mask array to keep track of the filtered values
    mask = np.ones_like(X, dtype=bool)

    # Sort the array X and retrieve the derivative values at those points
    sorted_indices = np.argsort(X)
    sorted_X = X[sorted_indices]
    sorted_dgdx = np.abs(dgdx(sorted_X))

    # Iterate over each value in sorted_X
    for i in range(len(sorted_X)):
        if mask[i]:
            # Find the indices of values within a distance of 15 from sorted_X[i]
            close_indices = np.where(np.abs(sorted_X - sorted_X[i]) < distance)[0]
            #if len(close_indices) > 1:
            #    print(X[close_indices])
            #    print(dgdx(X[close_indices]))
            # Find the index of the minimum value of dgdx within the close_indices
            min_index = np.argmin(sorted_dgdx[close_indices])

            # Update the mask to keep only the minimum value
            mask[close_indices] = False
            mask[close_indices[min_index]] = True
    # Return the filtered X values
    filtered_X = sorted_X[mask]
    return filtered_X

#Define a function to compute the peak and thrusts locations of the cls, taking ells to be >10 and < ell_max:
def find_spline_peaks(ells,cl, Neff_value, mode,raw_peaks=[],ell_min=20,ell_max=5000,distance_threshold_btw_peaks=20, distance_threshold_wtn_peak=15, Neff_fiducial=3.044):
    g=CubicSpline(ells,cl)
    dgdx=g.derivative()
    d2gdx2=dgdx.derivative()
    x_minmax=dgdx.roots()
    #print(x_minmax)
    #check values of x_minmax, if they are closer than 15, keep only the one with the lowest value of the derivative:
    x_minmax=filter_roots(x_minmax,dgdx,d2gdx2,distance_threshold_wtn_peak)
    #print(x_minmax)
    #Only keep values of x_minmax> ell_min and < ell_max:
    x_minmax=x_minmax[(x_minmax>ell_min) & (x_minmax<ell_max)]
    ##if mode=='raw' return the raw peaks and troughs; otherwise only keep the peaks that are within a distance 15 from those computed for the raw model:
    if mode!='raw':
        x_minmax_filter=[]
        for i in range(len(raw_peaks)):
            abs_diff_peaks=np.abs(x_minmax-raw_peaks[i])
            diff_peaks=x_minmax-raw_peaks[i]
            #get indices for which diff_peaks<distance_threshold_btw_peaks:
            indices=np.where(abs_diff_peaks<(distance_threshold_btw_peaks))[0]
            if len(indices)==1:
                #save the value of x_minmax[indices[0]] in x_minmax_filter:
                x_minmax_filter.append(x_minmax[indices[0]])
            if len(indices)>1:
                #only keep diff_peaks[indices]>0:
                if Neff_value>Neff_fiducial:
                    indices=indices[diff_peaks[indices]<0]
                if Neff_value<Neff_fiducial:
                    indices=indices[diff_peaks[indices]>0]
                x_minmax_filter.append(x_minmax[indices[np.argmin(abs_diff_peaks[indices])]])
        x_minmax=np.array(x_minmax_filter)
        if len(x_minmax) < len(raw_peaks):
            #print(x_minmax)
            for zero in raw_peaks:
                close_indices = np.where(np.abs(x_minmax - zero) < distance_threshold_btw_peaks)[0]
                if close_indices.size == 0:
                    search_range = np.linspace(zero- distance_threshold_btw_peaks, zero + distance_threshold_btw_peaks, 100)
                    derivative_values = dgdx(search_range)
                    min_index = np.argmin(derivative_values)
                    x_minmax = np.append(x_minmax, search_range[min_index])
            x_minmax=np.sort(x_minmax)
        elif len(x_minmax) > len(raw_peaks):
            print('Error: too many peaks found, algorithm needs to be improved')
    return x_minmax, g(x_minmax)

# $\mathcal{C}_\ell$ Computations

In [ ]:
Cls_Neffs=[]
Cls_LCDM=Cl_computer(common_settings,{'omega_cdm':omega_cdm_LCDM, 'N_ur':3.044})
for i in trange(len(N_eff_array)):
    ells_cosmo,dTT_cosmo,dEE_cosmo=Cl_computer(common_settings,{'omega_cdm':omega_cdm_array[i], 'N_ur':N_eff_array[i]})
    Cls_Neffs.append([ells_cosmo,dTT_cosmo,dEE_cosmo])
    del ells_cosmo, dTT_cosmo, dEE_cosmo
Cls_Neffs=np.array(Cls_Neffs)
np.save('Cls_Neffs',Cls_Neffs)

In [ ]:
Cls_YHes=[]

Cls_LCDM=Cl_computer(common_settings,{'omega_cdm':omega_cdm_LCDM, 'N_ur':3.044})
for i in trange(len(YHes)):
    ells_cosmo,dTT_cosmo,dEE_cosmo=Cl_computer(common_settings,{'omega_cdm':omega_cdm_LCDM, 'N_ur':3.044, 'YHe':YHes[i]})
    Cls_YHes.append([ells_cosmo,dTT_cosmo,dEE_cosmo])
    del ells_cosmo, dTT_cosmo, dEE_cosmo
Cls_YHes=np.array(Cls_YHes)
np.save('Cls_YHes',Cls_YHes)

In [ ]:
for j in trange(len(N_eff_array2)):
    YHe_tDs=np.load('YHe_data/class_YHe-kD_data_Neff'+str(int(N_eff_array2[j]*10))+'.npy',allow_pickle=True)
    YHe_tD_func=interp1d(YHe_tDs[:,1],YHe_tDs[:,0])
    YHe_vals[j]=YHe_tD_func(thD_fid/np.pi) #we are fixing \theta_d to its value from the fiducial cosmology
Cls_Neff_phase_shift=[]
Cls_LCDM=Cl_computer(common_settings,{'omega_cdm':omega_cdm_LCDM, 'N_ur':3.044})
for i in trange(len(N_eff_array2)):
    ells_cosmo,dTT_cosmo,dEE_cosmo=Cl_computer(common_settings,{'omega_cdm':omega_cdms_v2[i], 'N_ur':N_eff_array2[i], 'YHe':YHe_vals[i]})
    Cls_Neff_phase_shift.append([ells_cosmo,dTT_cosmo,dEE_cosmo])
    del ells_cosmo, dTT_cosmo, dEE_cosmo
Cls_Neff_phase_shift=np.array(Cls_Neff_phase_shift)
np.save('Cls_Neff_phase_shift',Cls_Neff_phase_shift)

# Figures

## $N_\nu$ Damping Effect: 

In [ ]:
#Load N_nu data:
Cls_Neffs=np.load('Cls_Neffs.npy')
#Load Planck data:
Planck_EE_spectrum =np.loadtxt('COM_PowerSpect_CMB-EE-binned_R3.02.txt', skiprows=1)
Planck_TT_spectrum =np.loadtxt('COM_PowerSpect_CMB-TT-binned_R3.01.txt', skiprows=1)

fig1=plt.figure(figsize=(3,3),dpi=300)
gs1 = GridSpec(20,20,figure=fig1)
ax1 = fig1.add_subplot(gs1[1:10,1:-3])
ax2 = fig1.add_subplot(gs1[10:-1,1:-3])
axDM = fig1.add_subplot(gs1[1:-1,-1])

color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array)))

cmap = plt.get_cmap('rainbow', len(Cls_Neffs))
# Normalizer
norm = mpl.colors.Normalize(vmin=0, vmax=np.max(N_eff_array))
# creating ScalarMappable
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

sm.set_array([])


# Use this ScalarMappable for the colorbar
plt.colorbar(sm, cax=axDM, ticks=[0, 3.044, np.max(N_eff_array)])

ax1.tick_params(axis='both', which='major', labelsize=6.5)
ax1.tick_params(axis='both', which='minor', labelsize=6.5)
ax2.tick_params(axis='both', which='major', labelsize=6.5)
ax2.tick_params(axis='both', which='minor', labelsize=6.5)

for i in range(len(N_eff_array)):
    ax1.plot(Cls_Neffs[i][0],Cls_Neffs[i][1],color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(Cls_Neffs[i][0],Cls_Neffs[i][2],color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

ax1.set_yscale('log')
ax2.set_yscale('log')
axDM.tick_params(axis='both', which='major', labelsize=6.5)
axDM.tick_params(axis='both', which='minor', labelsize=6.5)
axDM.set_xticks([])
# Move y-axis ticks and labels to the right
axDM.yaxis.tick_right()
axDM.yaxis.set_label_position("right")
#Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
axDM.axhline(3.044, color='black', linewidth=1)  # Add the horizontal line
plt.colorbar(sm,cax=axDM)
#axDM.set_yticks([0,0.1,0.2,0.3])
#axDM.set_yticklabels([r'$0.0$',r'$0.1$',r'$0.2$',r'$0.3$'],fontdict={'family':'serif','fontsize':4.5})
ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.7,ls='solid', label=r'$\Lambda$CDM')
ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.7,ls='solid')
ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)

ax1.set_ylim(5e-2,1e1)
ax1.set_yticks([1e-1,1,1e1])
ax1.set_xticklabels([r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
ax2.set_ylim(1e-4,6e-2)
ax2.set_yticks([3e-4,1e-3,1e-2])
ax2.set_xticklabels([r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$'],fontdict={'family':'serif','fontsize':5.5})
ax1.set_xlim(0,2500)
ax2.set_xlim(0,2500)
ax1.set_xticks([0,500,1000,1500,2000,2500])
ax2.set_xticks([0,500,1000,1500,2000,2500])
ax1.set_xticklabels([])
ax2.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$'],fontdict={'family':'serif','fontsize':5.5})
ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
ax1.yaxis.set_minor_formatter(NullFormatter())
ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
ax2.yaxis.set_minor_formatter(NullFormatter())
ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
axDM.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
ax1.legend(prop={'family':'serif','size':5.5})
ax1.text(200,1e-1,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b$ fixed',fontdict={'fontsize':6,'family':'serif'})
plt.savefig('Neff_gif/Dl_damping_Neff_tot.png',bbox_inches='tight',dpi=300)
plt.show()
plt.close()

In [ ]:

for i in trange(len(N_eff_array)):
    fig1=plt.figure(figsize=(3,3),dpi=300)
    gs1 = GridSpec(20,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-3])
    ax2 = fig1.add_subplot(gs1[10:-1,1:-3])
    axNeff = fig1.add_subplot(gs1[1:-1,-1])

    color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array)))

    cmap = plt.get_cmap('rainbow', len(Cls_Neffs))
    if i!=len(N_eff_array)-1:
        color_Neff[i+1:]=[1,1,1,1]
    
    custom_cmap= mpl.colors.ListedColormap(color_Neff)
    # Normalizer
    norm = mpl.colors.Normalize(vmin=0, vmax=np.max(N_eff_array))
    # creating ScalarMappable
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)

    sm.set_array([])
    # Use this ScalarMappable for the colorbar
    plt.colorbar(sm, cax=axNeff)

    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ax1.plot(Cls_Neffs[i][0],Cls_Neffs[i][1],color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(Cls_Neffs[i][0],Cls_Neffs[i][2],color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

    ax1.set_yscale('log')
    ax2.set_yscale('log')
    axNeff.tick_params(axis='both', which='major', labelsize=6.5)
    axNeff.tick_params(axis='both', which='minor', labelsize=6.5)
    axNeff.set_xticks([])
    # Move y-axis ticks and labels to the right
    axNeff.yaxis.tick_right()
    axNeff.yaxis.set_label_position("right")
    if i!=len(N_eff_array)-1:
        #Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
        axNeff.axhline(N_eff_array[i], color='black', linewidth=1)  # Add the horizontal line
    plt.colorbar(sm,cax=axNeff)
    #axDM.set_yticks([0,0.1,0.2,0.3])
    #axDM.set_yticklabels([r'$0.0$',r'$0.1$',r'$0.2$',r'$0.3$'],fontdict={'family':'serif','fontsize':4.5})
    ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.7,ls='solid', label=r'$\Lambda$CDM')
    ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.7,ls='solid')
    ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
    ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)

    ax1.set_ylim(5e-2,1e1)
    ax1.set_yticks([1e-1,1,1e1])
    ax1.set_xticklabels([r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
    ax2.set_ylim(1e-4,6e-2)
    ax2.set_yticks([3e-4,1e-3,1e-2])
    ax2.set_xticklabels([r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.set_xlim(0,2500)
    ax2.set_xlim(0,2500)
    ax1.set_xticks([0,500,1000,1500,2000,2500])
    ax2.set_xticks([0,500,1000,1500,2000,2500])
    ax1.set_xticklabels([])
    ax2.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax1.yaxis.set_minor_formatter(NullFormatter())
    ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax2.yaxis.set_minor_formatter(NullFormatter())
    ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    axNeff.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
    ax1.legend(prop={'family':'serif','size':5.5})
    ax1.text(200,1e-1,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b$ fixed',fontdict={'fontsize':6,'family':'serif'})
    plt.savefig('Neff_gif/Dl_damping_Neff_v'+str(i+1)+'.png',bbox_inches='tight',dpi=300)
    plt.close()
    del fig1

In [ ]:
from PIL import Image as Im

from IPython.display import Image, display

#################################################
##         Script to actualy create gif        ##
#################################################

images_Neff_damping = [Im.open('Neff_gif/Dl_damping_Neff_v'+str(i+1)+'.png') for i in range(len(N_eff_array))]

# Save the images as a GIF
images_Neff_damping[0].save('Neff_gif/Dl_Neff_damping.gif', save_all=True, append_images=images_Neff_damping[1:], optimize=False, duration=200, loop=0)
# Display the GIF in the notebook
display(Image('Neff_gif/Dl_Neff_damping.gif'))

## $Y_{\rm{He}}$ Damping Effect 

In [ ]:
#Load YHe data:
Cls_YHes=np.load('Cls_YHes.npy')

#Load Planck data:
Planck_EE_spectrum =np.loadtxt('COM_PowerSpect_CMB-EE-binned_R3.02.txt', skiprows=1)
Planck_TT_spectrum =np.loadtxt('COM_PowerSpect_CMB-TT-binned_R3.01.txt', skiprows=1)

fig1=plt.figure(figsize=(3,3),dpi=300)
gs1 = GridSpec(20,20,figure=fig1)
ax1 = fig1.add_subplot(gs1[1:10,1:-3])
ax2 = fig1.add_subplot(gs1[10:-1,1:-3])
axDM = fig1.add_subplot(gs1[1:-1,-1])

color_YHe= plt.cm.rainbow(np.linspace(0, 1, len(YHes)))

cmap = plt.get_cmap('rainbow', len(Cls_YHes))
# Normalizer
norm = mpl.colors.Normalize(vmin=np.min(YHes), vmax=np.max(YHes))
# creating ScalarMappable
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

sm.set_array([])


# Use this ScalarMappable for the colorbar
plt.colorbar(sm, cax=axDM)

ax1.tick_params(axis='both', which='major', labelsize=6.5)
ax1.tick_params(axis='both', which='minor', labelsize=6.5)
ax2.tick_params(axis='both', which='major', labelsize=6.5)
ax2.tick_params(axis='both', which='minor', labelsize=6.5)

for i in range(len(YHes)):
    ax1.plot(Cls_YHes[i][0],Cls_YHes[i][1],color=color_YHe[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(Cls_YHes[i][0],Cls_YHes[i][2],color=color_YHe[i],alpha=0.9,lw=0.9,ls='solid')

ax1.set_yscale('log')
ax2.set_yscale('log')
axDM.tick_params(axis='both', which='major', labelsize=6.5)
axDM.tick_params(axis='both', which='minor', labelsize=6.5)
axDM.set_xticks([])
# Move y-axis ticks and labels to the right
axDM.yaxis.tick_right()
axDM.yaxis.set_label_position("right")
#Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
axDM.axhline(0.2456, color='black', linewidth=1)  # Add the horizontal line
plt.colorbar(sm,cax=axDM)
axDM.set_yticks([0.1,0.2,0.3,0.4,0.5])
axDM.set_yticklabels([r'$0.1$',r'$0.2$',r'$0.3$',r'$0.4$',r'$0.5$'],fontdict={'family':'serif','fontsize':4.5})
ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.7,ls='solid', label=r'$\Lambda$CDM')
ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.7,ls='solid')
ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)

ax1.set_ylim(5e-2,1e1)
ax1.set_yticks([1e-1,1,1e1])
ax1.set_xticklabels([r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
ax2.set_ylim(1e-4,6e-2)
ax2.set_yticks([3e-4,1e-3,1e-2])
ax2.set_xticklabels([r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$'],fontdict={'family':'serif','fontsize':5.5})
ax1.set_xlim(0,2500)
ax2.set_xlim(0,2500)
ax1.set_xticks([0,500,1000,1500,2000,2500])
ax2.set_xticks([0,500,1000,1500,2000,2500])
ax1.set_xticklabels([])
ax2.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$'],fontdict={'family':'serif','fontsize':5.5})
ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
ax1.yaxis.set_minor_formatter(NullFormatter())
ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
ax2.yaxis.set_minor_formatter(NullFormatter())
ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
axDM.set_xlabel(r"$Y_{\rm{He}}$",fontdict={'fontsize':7.5,'family':'serif'})
ax1.legend(prop={'family':'serif','size':5.5})
ax1.text(200,1e-1,r'$\theta_s,\,\omega_b$ fixed',fontdict={'fontsize':6,'family':'serif'})
plt.savefig('Neff_gif/Dl_damping_YHe_tot.png',bbox_inches='tight',dpi=300)
plt.show()
plt.close()

In [ ]:
for i in trange(len(YHes)):
    fig1=plt.figure(figsize=(3,3),dpi=300)
    gs1 = GridSpec(20,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-3])
    ax2 = fig1.add_subplot(gs1[10:-1,1:-3])
    axDM = fig1.add_subplot(gs1[1:-1,-1])

    color_YHe= plt.cm.rainbow(np.linspace(0, 1, len(YHes)))

    cmap = plt.get_cmap('rainbow', len(Cls_YHes))
    # Normalizer
    norm = mpl.colors.Normalize(vmin=np.min(YHes), vmax=np.max(YHes))
    if i!=len(YHes)-1:
        color_YHe[i+1:]=[1,1,1,1]
    custom_cmap= mpl.colors.ListedColormap(color_YHe)
    # Normalizer
    norm = mpl.colors.Normalize(vmin=np.min(YHes), vmax=np.max(YHes))
    # creating ScalarMappable
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])
    # Use this ScalarMappable for the colorbar
    plt.colorbar(sm, cax=axDM)
    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ax1.plot(Cls_YHes[i][0],Cls_YHes[i][1],color=color_YHe[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(Cls_YHes[i][0],Cls_YHes[i][2],color=color_YHe[i],alpha=0.9,lw=0.9,ls='solid')

    ax1.set_yscale('log')
    ax2.set_yscale('log')
    axDM.tick_params(axis='both', which='major', labelsize=6.5)
    axDM.tick_params(axis='both', which='minor', labelsize=6.5)
    axDM.set_xticks([])
    # Move y-axis ticks and labels to the right
    axDM.yaxis.tick_right()
    axDM.yaxis.set_label_position("right")
    #Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
    if i!=len(YHes)-1:
        axDM.axhline(YHes[i], color='black', linewidth=1)  # Add the horizontal line
    plt.colorbar(sm,cax=axDM)
    axDM.set_yticks([0.1,0.2,0.3,0.4,0.5])
    axDM.set_yticklabels([r'$0.1$',r'$0.2$',r'$0.3$',r'$0.4$',r'$0.5$'],fontdict={'family':'serif','fontsize':4.5})
    ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.7,ls='solid', label=r'$\Lambda$CDM')
    ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.7,ls='solid')
    ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
    ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)

    ax1.set_ylim(5e-2,1e1)
    ax1.set_yticks([1e-1,1,1e1])
    ax1.set_xticklabels([r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
    ax2.set_ylim(1e-4,6e-2)
    ax2.set_yticks([3e-4,1e-3,1e-2])
    ax2.set_xticklabels([r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.set_xlim(0,2500)
    ax2.set_xlim(0,2500)
    ax1.set_xticks([0,500,1000,1500,2000,2500])
    ax2.set_xticks([0,500,1000,1500,2000,2500])
    ax1.set_xticklabels([])
    ax2.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax1.yaxis.set_minor_formatter(NullFormatter())
    ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax2.yaxis.set_minor_formatter(NullFormatter())
    ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    axDM.set_xlabel(r"$Y_{\rm{He}}$",fontdict={'fontsize':7.5,'family':'serif'})
    ax1.legend(prop={'family':'serif','size':5.5})
    ax1.text(200,1e-1,r'$\theta_s,\,\omega_b$ fixed',fontdict={'fontsize':6,'family':'serif'})
    plt.savefig('Neff_gif/Dl_damping_YHe_v'+str(i+1)+'.png',bbox_inches='tight',dpi=300)
    plt.close()
    del fig1

In [ ]:
#################################################
##         Script to actualy create gif        ##
#################################################

images_YHes = [Im.open('Neff_gif/Dl_damping_YHe_v'+str(i+1)+'.png') for i in range(len(YHes))]

# Save the images as a GIF
images_YHes[0].save('Neff_gif/Dl_YHe_damping.gif', save_all=True, append_images=images_YHes[1:], optimize=False, duration=200, loop=0)
# Display the GIF in the notebook
display(Image('Neff_gif/Dl_YHe_damping.gif'))

## Phase shift effect on TT spectrum

In [ ]:
Cls_Neff_phase_shift=np.load('Cls_Neff_phase_shift.npy')

afitTT=0.68
kfitTT=1.3

fig1=plt.figure(figsize=(3,3),dpi=300)
gs1 = GridSpec(20,20,figure=fig1)
ax1 = fig1.add_subplot(gs1[1:10,1:-3])
ax2 = fig1.add_subplot(gs1[12:-1,1:-3])
axDM = fig1.add_subplot(gs1[1:-1,-1])

color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array2)))

cmap = plt.get_cmap('rainbow', len(Cls_Neff_phase_shift))
# Normalizer
norm = mpl.colors.Normalize(vmin=np.min(N_eff_array2), vmax=np.max(N_eff_array2))
# creating ScalarMappable
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

sm.set_array([])


# Use this ScalarMappable for the colorbar
plt.colorbar(sm, cax=axDM, ticks=[0, 3.044, np.max(N_eff_array2)])

ax1.tick_params(axis='both', which='major', labelsize=6.5)
ax1.tick_params(axis='both', which='minor', labelsize=6.5)
ax2.tick_params(axis='both', which='major', labelsize=6.5)
ax2.tick_params(axis='both', which='minor', labelsize=6.5)
ells_LCDM=Cls_LCDM[0]
dTT_LCDM=Cls_LCDM[1]
kTT_LCDM=Kll(ells_LCDM,dTT_LCDM,afitTT,thD_fid,kfitTT)
ell_pksTT_lcdm,Kl_pksTT_lcdm=find_spline_peaks(ells_LCDM,kTT_LCDM,3.044,'raw',ell_min=100,ell_max=2500)
peak4TT_lcdm=Kl_pksTT_lcdm[6]

for i in range(len(N_eff_array2)):
    ells_Neff=Cls_Neff_phase_shift[i][0]
    dll_Neff=Cls_Neff_phase_shift[i][1]
    kTT_Neff=Kll(ells_Neff,dll_Neff,afitTT,thD_fid,kfitTT)
    ell_pksTT_Neff,Kl_pksTT_Neff=find_spline_peaks(ells_Neff,kTT_Neff,N_eff_array2[i],'raw',ell_min=100,ell_max=2500)
    peak4TT_Neff=Kl_pksTT_Neff[6]
    ax1.plot(ells_Neff,kTT_Neff*peak4TT_lcdm/peak4TT_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(ells_Neff[(ells_Neff>=950) & (ells_Neff<=1250)],kTT_Neff[(ells_Neff>=950) & (ells_Neff<=1250)]*peak4TT_lcdm/peak4TT_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

ax1.plot(ells_LCDM,kTT_LCDM,color='k',alpha=0.9,lw=0.3,ls='solid')
ax2.plot(ells_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],kTT_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],color='k',alpha=0.9,lw=0.4,ls='solid',label='$\Lambda$CDM')

axDM.tick_params(axis='both', which='major', labelsize=6.5)
axDM.tick_params(axis='both', which='minor', labelsize=6.5)
axDM.set_xticks([])
# Move y-axis ticks and labels to the right
axDM.yaxis.tick_right()
axDM.yaxis.set_label_position("right")
#Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
axDM.axhline(3.044, color='black', linewidth=1)  # Add the horizontal line
plt.colorbar(sm,cax=axDM)

ax1.set_xlim(0,2500)
ax2.set_xlim(950,1250)
ax1.set_xticks([0,500,1000,1500,2000,2500])
ax1.set_ylim(0,8)
ax2.set_ylim(3.5,6)
ax2.set_yticks([4,5,6])
ax1.set_yticks([0,2,4,6,8])
#ax1.set_ylabel(r"$\mathcal{K}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
#ax2.set_ylabel(r"$\mathcal{K}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
axDM.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
ax2.legend(prop={'family':'serif','size':5.5})
ax1.text(900,1,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b,\,\theta_d,\,A$ fixed',fontdict={'fontsize':6,'family':'serif'})
ax2.text(900,6.6,r'$\mathcal{K}^{TT}_\ell$ [$10^3 \mu\mathrm{K}^2$]',rotation=90,va='center',fontdict={'fontsize':9,'family':'serif'})
plt.savefig('Neff_gif/kTT_phase_shift_Neff_tot_lens.png',bbox_inches='tight',dpi=300)
plt.show()
plt.close()

In [ ]:
for i in trange(len(N_eff_array2)):
    fig1=plt.figure(figsize=(3,3),dpi=300)
    gs1 = GridSpec(20,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-3])
    ax2 = fig1.add_subplot(gs1[12:-1,1:-3])
    axDM = fig1.add_subplot(gs1[1:-1,-1])

    color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array2)))

    cmap = plt.get_cmap('rainbow', len(Cls_Neff_phase_shift))
    if i!=len(N_eff_array2)-1:
        color_Neff[i+1:]=[1,1,1,1]

    custom_cmap= mpl.colors.ListedColormap(color_Neff)
    # Normalizer
    norm = mpl.colors.Normalize(vmin=np.min(N_eff_array2), vmax=np.max(N_eff_array2))
    # creating ScalarMappable
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])

    # Use this ScalarMappable for the colorbar
    plt.colorbar(sm, cax=axDM)

    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ells_LCDM=Cls_LCDM[0]
    dTT_LCDM=Cls_LCDM[1]
    kTT_LCDM=Kll(ells_LCDM,dTT_LCDM,afitTT,thD_fid,kfitTT)
    ell_pksTT_lcdm,Kl_pksTT_lcdm=find_spline_peaks(ells_LCDM,kTT_LCDM,3.044,'raw',ell_min=100,ell_max=2500)
    peak4TT_lcdm=Kl_pksTT_lcdm[6]


    ells_Neff=Cls_Neff_phase_shift[i][0]
    dll_Neff=Cls_Neff_phase_shift[i][1]
    kTT_Neff=Kll(ells_Neff,dll_Neff,afitTT,thD_fid,kfitTT)
    ell_pksTT_Neff,Kl_pksTT_Neff=find_spline_peaks(ells_Neff,kTT_Neff,N_eff_array2[i],'raw',ell_min=100,ell_max=2500)
    peak4TT_Neff=Kl_pksTT_Neff[6]
    ax1.plot(ells_Neff,kTT_Neff*peak4TT_lcdm/peak4TT_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(ells_Neff[(ells_Neff>=950) & (ells_Neff<=1250)],kTT_Neff[(ells_Neff>=950) & (ells_Neff<=1250)]*peak4TT_lcdm/peak4TT_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

    ax1.plot(ells_LCDM,kTT_LCDM,color='k',alpha=0.9,lw=0.3,ls='solid')
    ax2.plot(ells_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],kTT_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],color='k',alpha=0.9,lw=0.4,ls='solid',label='$\Lambda$CDM')

    axDM.tick_params(axis='both', which='major', labelsize=6.5)
    axDM.tick_params(axis='both', which='minor', labelsize=6.5)
    axDM.set_xticks([])
    # Move y-axis ticks and labels to the right
    axDM.yaxis.tick_right()
    axDM.yaxis.set_label_position("right")
    #Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
    plt.colorbar(sm,cax=axDM)

    ax1.set_xlim(0,2500)
    ax2.set_xlim(950,1250)
    ax1.set_xticks([0,500,1000,1500,2000,2500])
    ax1.set_ylim(0,8)
    ax2.set_ylim(3.5,6)
    ax2.set_yticks([4,5,6])
    ax1.set_yticks([0,2,4,6,8])
    ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    axDM.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
    ax2.legend(prop={'family':'serif','size':5.5})
    ax1.text(900,1,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b,\,\theta_d,\,A$ fixed',fontdict={'fontsize':6,'family':'serif'})
    ax2.text(900,6.6,r'$\mathcal{K}^{TT}_\ell$ [$10^3 \mu\mathrm{K}^2$]',rotation=90,va='center',fontdict={'fontsize':9,'family':'serif'})
    plt.savefig('Neff_gif/kTT_phase_shift_Neff_v'+str(i+1)+'_lens.png',bbox_inches='tight',dpi=300)
    plt.close()
    del fig1

In [ ]:
#################################################
##         Script to actualy create gif        ##
#################################################

images_Neff_phase_shift_TT = [Im.open('Neff_gif/kTT_phase_shift_Neff_v'+str(i+1)+'_lens.png') for i in range(len(N_eff_array2))]

# Save the images as a GIF
images_Neff_phase_shift_TT[0].save('Neff_gif/kTT_phase_shift_Neff_lens.gif', save_all=True, append_images=images_Neff_phase_shift_TT[1:], optimize=False, duration=200, loop=0)
# Display the GIF in the notebook
display(Image('Neff_gif/kTT_phase_shift_Neff_lens.gif'))

## Phase shift effect on EE spectrum

In [ ]:
Cls_Neff_phase_shift=np.load('Cls_Neff_phase_shift.npy')

afitEE=0.5
kfitEE=1.3

fig1=plt.figure(figsize=(3,3),dpi=300)
gs1 = GridSpec(20,20,figure=fig1)
ax1 = fig1.add_subplot(gs1[1:10,1:-3])
ax2 = fig1.add_subplot(gs1[12:-1,1:-3])
axDM = fig1.add_subplot(gs1[1:-1,-1])

color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array2)))

cmap = plt.get_cmap('rainbow', len(Cls_Neffs))
# Normalizer
norm = mpl.colors.Normalize(vmin=0, vmax=np.max(N_eff_array2))
# creating ScalarMappable
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

sm.set_array([])


# Use this ScalarMappable for the colorbar
plt.colorbar(sm, cax=axDM, ticks=[0, 3.044, np.max(N_eff_array2)])

ax1.tick_params(axis='both', which='major', labelsize=6.5)
ax1.tick_params(axis='both', which='minor', labelsize=6.5)
ax2.tick_params(axis='both', which='major', labelsize=6.5)
ax2.tick_params(axis='both', which='minor', labelsize=6.5)
ells_LCDM=Cls_LCDM[0]
dEE_LCDM=Cls_LCDM[2]
kEE_LCDM=Kll(ells_LCDM,dEE_LCDM,afitEE,thD_fid,kfitEE)
ell_pksEE_lcdm,Kl_pksEE_lcdm=find_spline_peaks(ells_LCDM,kEE_LCDM,3.044,'raw',ell_min=100,ell_max=2500)
peak4EE_lcdm=Kl_pksEE_lcdm[6]

for i in range(len(N_eff_array2)):
    ells_Neff=Cls_Neff_phase_shift[i][0]
    dll_Neff=Cls_Neff_phase_shift[i][2]
    kEE_Neff=Kll(ells_Neff,dll_Neff,afitEE,thD_fid,kfitEE)
    ell_pksEE_Neff,Kl_pksEE_Neff=find_spline_peaks(ells_Neff,kEE_Neff,N_eff_array2[i],'raw',ell_min=100,ell_max=2500)
    peak4EE_Neff=Kl_pksEE_Neff[6]
    ax1.plot(ells_Neff,kEE_Neff*peak4EE_lcdm/peak4EE_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(ells_Neff[(ells_Neff>=950) & (ells_Neff<=1250)],kEE_Neff[(ells_Neff>=950) & (ells_Neff<=1250)]*peak4EE_lcdm/peak4EE_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

ax1.plot(ells_LCDM,kEE_LCDM,color='k',alpha=0.9,lw=0.3,ls='solid')
ax2.plot(ells_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],kEE_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],color='k',alpha=0.9,lw=0.4,ls='solid',label='$\Lambda$CDM')

axDM.tick_params(axis='both', which='major', labelsize=6.5)
axDM.tick_params(axis='both', which='minor', labelsize=6.5)
axDM.set_xticks([])
# Move y-axis ticks and labels to the right
axDM.yaxis.tick_right()
axDM.yaxis.set_label_position("right")
#Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
axDM.axhline(3.044, color='black', linewidth=1)  # Add the horizontal line
plt.colorbar(sm,cax=axDM)

ax1.set_ylim(0,.15)
ax2.set_ylim(0.03,0.13)
ax1.set_yticks([0.0,0.05,0.1,0.15])
ax2.set_yticks([0.05,0.1])
ax1.set_xlim(0,2500)
ax2.set_xlim(950,1250)
ax1.set_xticks([0,500,1000,1500,2000,2500])
ax2.set_xticks([950,1000,1050,1100,1150,1200,1250])
ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
axDM.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
ax2.legend(prop={'family':'serif','size':5.5})
ax1.text(900,0.02,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b,\,\theta_d,\,A$ fixed',fontdict={'fontsize':6,'family':'serif'})
ax2.text(880,.15,r'$\mathcal{K}^{EE}_\ell$ [$10^3 \mu\mathrm{K}^2$]',rotation=90,va='center',fontdict={'fontsize':9,'family':'serif'})
plt.savefig('Neff_gif/kEE_phase_shift_Neff_tot_lens.png',bbox_inches='tight',dpi=300)
plt.show()
plt.close()

In [ ]:
for i in trange(len(N_eff_array2)):
    
    fig1=plt.figure(figsize=(3,3),dpi=300)
    gs1 = GridSpec(20,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-3])
    ax2 = fig1.add_subplot(gs1[12:-1,1:-3])
    axDM = fig1.add_subplot(gs1[1:-1,-1])

    color_Neff= plt.cm.rainbow(np.linspace(0, 1, len(N_eff_array2)))

    cmap = plt.get_cmap('rainbow', len(Cls_Neff_phase_shift))
    if i!=len(N_eff_array)-1:
        color_Neff[i+1:]=[1,1,1,1]
    
    custom_cmap= mpl.colors.ListedColormap(color_Neff)
    # Normalizer
    norm = mpl.colors.Normalize(vmin=np.min(N_eff_array2), vmax=np.max(N_eff_array2))
    # creating ScalarMappable
    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
    sm.set_array([])
    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ells_LCDM=Cls_LCDM[0]
    dEE_LCDM=Cls_LCDM[2]
    kEE_LCDM=Kll(ells_LCDM,dEE_LCDM,afitEE,thD_fid,kfitEE)
    ell_pksEE_lcdm,Kl_pksEE_lcdm=find_spline_peaks(ells_LCDM,kEE_LCDM,3.044,'raw',ell_min=100,ell_max=2500)
    peak4EE_lcdm=Kl_pksEE_lcdm[6]


    ells_Neff=Cls_Neff_phase_shift[i][0]
    dll_Neff=Cls_Neff_phase_shift[i][2]
    kEE_Neff=Kll(ells_Neff,dll_Neff,afitEE,thD_fid,kfitEE)
    ell_pksEE_Neff,Kl_pksEE_Neff=find_spline_peaks(ells_Neff,kEE_Neff,N_eff_array2[i],'raw',ell_min=100,ell_max=2500)
    peak4EE_Neff=Kl_pksEE_Neff[6]
    ax1.plot(ells_Neff,kEE_Neff*peak4EE_lcdm/peak4EE_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')
    ax2.plot(ells_Neff[(ells_Neff>=950) & (ells_Neff<=1250)],kEE_Neff[(ells_Neff>=950) & (ells_Neff<=1250)]*peak4EE_lcdm/peak4EE_Neff,color=color_Neff[i],alpha=0.9,lw=0.9,ls='solid')

    ax1.plot(ells_LCDM,kEE_LCDM,color='k',alpha=0.9,lw=0.3,ls='solid')
    ax2.plot(ells_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],kEE_LCDM[(ells_LCDM>=950) & (ells_LCDM<=1250)],color='k',alpha=0.9,lw=0.4,ls='solid',label='$\Lambda$CDM')

    axDM.tick_params(axis='both', which='major', labelsize=6.5)
    axDM.tick_params(axis='both', which='minor', labelsize=6.5)
    axDM.set_xticks([])
    # Move y-axis ticks and labels to the right
    axDM.yaxis.tick_right()
    axDM.yaxis.set_label_position("right")
    #Create Horizontal bar on axDM indicating LCDM value of omega_CDM:
    plt.colorbar(sm,cax=axDM)

    ax1.set_ylim(0,.15)
    ax2.set_ylim(0.03,0.13)
    ax1.set_yticks([0.0,0.05,0.1,0.15])
    ax2.set_yticks([0.05,0.1])
    ax1.set_xlim(0,2500)
    ax2.set_xlim(950,1250)
    ax1.set_xticks([0,500,1000,1500,2000,2500])
    ax2.set_xticks([950,1000,1050,1100,1150,1200,1250])
    ax2.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    axDM.set_xlabel(r"$N_{\nu}$",fontdict={'fontsize':7.5,'family':'serif'})
    ax2.legend(prop={'family':'serif','size':5.5})
    ax1.text(900,0.02,r'$\theta_s,\,a_{\rm{eq}},\,\omega_b,\,\theta_d,\,A$ fixed',fontdict={'fontsize':6,'family':'serif'})
    ax2.text(880,.15,r'$\mathcal{K}^{EE}_\ell$ [$10^3 \mu\mathrm{K}^2$]',rotation=90,va='center',fontdict={'fontsize':9,'family':'serif'})
    plt.savefig('Neff_gif/kEE_phase_shift_Neff_v'+str(i+1)+'_lens.png',bbox_inches='tight',dpi=300)
    plt.close()
    del fig1

In [ ]:
#################################################
##         Script to actualy create gif        ##
#################################################

images_Neff_phase_shift_EE = [Im.open('Neff_gif/kEE_phase_shift_Neff_v'+str(i+1)+'_lens.png') for i in range(len(N_eff_array2))]

# Save the images as a GIF
images_Neff_phase_shift_EE[0].save('Neff_gif/kEE_phase_shift_Neff_lens.gif', save_all=True, append_images=images_Neff_phase_shift_EE[1:], optimize=False, duration=200, loop=0)
# Display the GIF in the notebook
display(Image('Neff_gif/kEE_phase_shift_Neff_lens.gif'))